In [2]:
import pandas as pd
import numpy as np
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import os

In [3]:
word2TagDictLanguage = {}
word2TagDictNER = {}
dictTagsLanguage = {}
dictTagsNER = {}
data = []

#TODO: Read both files here
#TODO: there will be 2 separate tag doctionaries
#TODO: word vocab dictionary will be common

## Language Data 
with open('final.tsv', 'r') as train_data_language:
    tagLine = train_data_language.readline()
    while tagLine:
        tokens = tagLine.split()
        if len(tokens) == 6:
            if tokens[0] in dictTagsLanguage:
                dictTagsLanguage[tokens[0]][tokens[4]] = tokens[5]
            else:
                dictTagsLanguage[tokens[0]] = {}
                dictTagsLanguage[tokens[0]][tokens[4]] = tokens[5]

            word2TagDictLanguage[tokens[4]] = tokens[5]
        tagLine = train_data_language.readline()

## NER data
with open('data.tsv', 'r') as train_data_NER:
    tagLine = train_data_NER.readline()
    while tagLine:
        tokens = tagLine.split()
        if len(tokens) == 6:
            if tokens[0] in dictTagsNER:
                dictTagsNER[tokens[0]][tokens[4]] = tokens[5]
            else:
                dictTagsNER[tokens[0]] = {}
                dictTagsNER[tokens[0]][tokens[4]] = tokens[5]

            word2TagDictNER[tokens[4]] = tokens[5]
        tagLine = train_data_NER.readline()


In [4]:
#TODO: Do this for each of the two datsets

dataNER = []
dataLanguage = []
for tweet in dictTagsLanguage:
    sentence = dictTagsLanguage[tweet]
    listTup = [(k, v) for k, v in sentence.items()]
    dataLanguage.append(listTup)
    
for tweet in dictTagsNER:
    sentence = dictTagsNER[tweet]
    listTup = [(k, v) for k, v in sentence.items()]
    dataNER.append(listTup)

In [5]:
#TODO: tags will be separate for each type
#TODO: words will be combined

wordsLanguage = set(word2TagDictLanguage.keys())
wordsNER = set(word2TagDictNER.keys())
tagsLanguage = list(set(word2TagDictLanguage.values()))
tagsNER = list(set(word2TagDictNER.values()))


In [6]:
tagsLanguage

['ambiguous', 'other', 'mixed', 'lang1', 'ne', 'lang2']

In [7]:
tagsNER

['I-GROUP',
 'B-LOC',
 'B-TITLE',
 'I-PROD',
 'I-ORG',
 'B-PROD',
 'B-EVENT',
 'I-OTHER',
 'I-LOC',
 'I-EVENT',
 'I-TIME',
 'I-PER',
 'O',
 'B-PER',
 'B-TIME',
 'B-ORG',
 'B-GROUP',
 'B-OTHER',
 'I-TITLE']

In [8]:
num_tags_language = len(tagsLanguage)
num_words_language = len(wordsLanguage)
num_words_NER = len(wordsNER)
num_tags_NER = len(tagsNER)

In [9]:
num_words_NER

62515

In [10]:
words = list(wordsLanguage.union(wordsNER))
num_words = len(words)

In [11]:
#TODO: words are combined, so only 1 dictionary
#TODO: tags are 2 sets, so 2 separate dictionaries
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tagLang2idx = {t: i for i, t in enumerate(tagsLanguage)}
tagNER2idx = {t: i for i, t in enumerate(tagsNER)}

In [12]:
#TODO: create X_ner, X_lang for the two separate datastes - same word2idx is used
from keras.preprocessing.sequence import pad_sequences
XLang = [[word2idx[w[0]] for w in s] for s in dataLanguage]
XNER = [[word2idx[w[0]] for w in s] for s in dataNER]

In [13]:
XLang = pad_sequences(maxlen=max_len, sequences=XLang, padding="post", value=num_words-1)
XNER = pad_sequences(maxlen=max_len, sequences=XNER, padding="post", value=num_words-1)

In [14]:
#TODO: create y_ner, y_lang for the two separate datastes - each has its own tag2idx
yLang = [[tagLang2idx[w[1]] for w in s] for s in dataLanguage]
yLang = pad_sequences(maxlen=max_len, sequences=yLang, padding="post", value=tagLang2idx["other"])
yNER = [[tagNER2idx[w[1]] for w in s] for s in dataNER]
yNER = pad_sequences(maxlen=max_len, sequences=yNER, padding="post", value=tagNER2idx["O"])


In [15]:
from keras.utils import to_categorical
yLang = [to_categorical(i, num_classes=num_tags_language) for i in yLang]
yNER = [to_categorical(i, num_classes=num_tags_NER) for i in yNER]

In [16]:
from sklearn.model_selection import train_test_split
X_lang_tr, X_lang_te, y_lang_tr, y_lang_te = train_test_split(XLang, yLang, test_size=0.1)
X_ner_tr, X_ner_te, y_ner_tr, y_ner_te = train_test_split(XNER, yNER, test_size=0.1)


In [2]:
import os
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [17]:
# input = Input(shape=(max_len,))
# model = Embedding(input_dim=num_words + 1, output_dim=50,
#                   input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
# model = Bidirectional(LSTM(units=50, return_sequences=True,
#                            recurrent_dropout=0.1))(model)  # variational biLSTM
# model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
# crf = CRF(num_tags)  # CRF layer
# out = crf(model)  # output

In [18]:
## biLSTM for language identification 
#TODO give a name to the model. will output results to a file based on model_name
model_name="default"
input = Input(shape=(max_len,))
model = Embedding(input_dim=num_words, output_dim={{choice([50, 100, 200])}}, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units={{choice([50, 100, 200])}}, return_sequences=True, recurrent_dropout=0.1))(model)
out_1 = TimeDistributed(Dense(num_tags_language, activation="softmax"))(model)  # softmax output layer
out_2 = TimeDistributed(Dense(num_tags_NER, activation="softmax"))(model)  # softmax output layer

In [19]:
model_1 = Model(input, out_1)

In [20]:
model_2 = Model(input, out_2)

In [21]:
# model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [21]:
model_1.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [22]:
model_2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [23]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
callbacksLanguage = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best_cs_joint_lang_model.h5', monitor='val_loss', save_best_only=True)]

callbacksNER = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best_cs_joint_ner_model.h5', monitor='val_loss', save_best_only=True)]


In [24]:
num_epochs = 15
for epoch in range(num_epochs):
    print("epoch = ", epoch)
    history_1 = model_1.fit(X_lang_tr, np.array(y_lang_tr), batch_size=32, callbacks=callbacksLanguage, epochs=1, validation_split=0.1, verbose=1)
    history_2 = model_2.fit(X_ner_tr, np.array(y_ner_tr), batch_size=32, callbacks=callbacksNER,epochs=1, validation_split=0.1, verbose=1)

epoch =  0
Train on 7072 samples, validate on 786 samples
Epoch 1/1
7072/7072 [==============================] - 25s 4ms/step - loss: 0.2670 - acc: 0.9220 - val_loss: 0.1020 - val_acc: 0.9679
Train on 29897 samples, validate on 3322 samples
Epoch 1/1
29897/29897 [==============================] - 102s 3ms/step - loss: 0.0510 - acc: 0.9915 - val_loss: 0.0239 - val_acc: 0.9954
epoch =  1
Train on 7072 samples, validate on 786 samples
Epoch 1/1
7072/7072 [==============================] - 24s 3ms/step - loss: 0.0790 - acc: 0.9757 - val_loss: 0.0465 - val_acc: 0.9872
Train on 29897 samples, validate on 3322 samples
Epoch 1/1
29897/29897 [==============================] - 101s 3ms/step - loss: 0.0189 - acc: 0.9957 - val_loss: 0.0193 - val_acc: 0.9958
epoch =  2
Train on 7072 samples, validate on 786 samples
Epoch 1/1
7072/7072 [==============================] - 24s 3ms/step - loss: 0.0467 - acc: 0.9868 - val_loss: 0.0381 - val_acc: 0.9899
Train on 29897 samples, validate on 3322 samples
Epo

In [25]:
#history = model_1.fit(X_lang_tr, np.array(y_lang_tr), batch_size=32, epochs=1, validation_split=0.1, verbose=1)
#history_2 = model_2.fit(X_tr, np.array(y_tr), batch_size=32, epochs=1, validation_split=0.1, verbose=1)

In [ ]:
#TODO: Need to load the best model. Currently the last epoch model is running
#TODO: For this, save the model while training - at every epoch
#TODO: And select model based on validation loss
#TODO: We can consider loading different model states for language and different for ner

In [25]:
# Evaluate language
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

from keras.models import load_model
model_1 = load_model('best_cs_joint_lang_model.h5')
test_pred = model_1.predict(X_lang_te, verbose=1)

idx2tag = {i: w for w, i in tagLang2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "other"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_lang_te)

print(classification_report(test_labels, pred_labels))
#TODO - print results to a file
# name using model_name

874/874 [==============================] - 1s 1ms/step
             precision    recall  f1-score   support

         ne       0.69      0.57      0.62       203
      lang2       0.68      0.64      0.66       667
      other       0.86      0.86      0.86      1888
      lang1       0.71      0.74      0.73      1117
  ambiguous       0.00      0.00      0.00        27
      mixed       0.00      0.00      0.00         1

avg / total       0.77      0.77      0.77      3903



In [26]:
# Evaluate ner
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

model_2 = load_model('best_cs_joint_ner_model.h5')
test_pred = model_2.predict(X_ner_te, verbose=1)

idx2tag = {i: w for w, i in tagNER2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "other"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_ner_te)

print(classification_report(test_labels, pred_labels))
#TODO - print results to a file
# name using model_name

3691/3691 [==============================] - 4s 1ms/step
             precision    recall  f1-score   support

       TIME       0.00      0.00      0.00        52
        ORG       0.00      0.00      0.00        54
       PROD       0.72      0.18      0.29        99
        PER       0.48      0.47      0.48       370
      GROUP       0.00      0.00      0.00        77
      OTHER       0.00      0.00      0.00        31
      TITLE       0.12      0.13      0.13        52
        LOC       0.53      0.45      0.49       185
      EVENT       0.00      0.00      0.00        15

avg / total       0.38      0.30      0.32       935

